## 1st: create a folder

In [ ]:
import datetime
import os
import sys
from utilities import make_folder, int_ask

# Set the project namea
# Set the project namea
user_list = os.listdir('/Users/lznus/Desktop/Data-Keithley/')
for i, user in enumerate(user_list):
    print(f'{i+1:02}: {user}')
user_ID = int(input('Select your user ID'))
user_name = user_list[user_ID-1]
project_name = input('Enter project name: ')
while project_name == '':
    print('Error: Project name cannot be empty')
    project_name = input('Enter project name: ')

# Get the date
today = datetime.datetime.today().strftime('%y%m%d')
project_dir = f"/Users/lznus/Desktop/Data-Keithley/{user_name}/{today}_tI-PULSE_{project_name}"

# Create the project directory if it does not exist
make_folder(project_dir)
# Create the project directory if it does not exist
fig_dir = f"/Users/lznus/Desktop/Data-Keithley/{user_name}/{today}_tI-PULSE_{project_name}/figures"
make_folder(fig_dir)

## 2nd: set parameters and make a file

In [ ]:
# Define parameters
KEITHLEY_ADDRESS = 'USB0::0x05E6::0x2450::04491080::INSTR'
source_voltage = -4.6 # V
delay_time = 10 #s
t_ON = 0.01 #s
duration = 560 # sec

file_name = input(f'Enter the file name [XXX], it will be "{project_dir}/XXX.csv"\n  [XXX] = ')
filename = os.path.join(project_dir, file_name + '.csv')

# Check if the project directory exists
overwrite = '' #initialize
if os.path.exists(filename):
    print(f'\033[33m\nThe file "{file_name}.csv" already exists.')
    overwrite = int_ask('Do you want to overwrite it? Yes(0) or No(1)\n')
    if overwrite == 0:
        print('\033[33mOkay, the program will overwrite the file.\n\033[33m')
    elif overwrite == 1:
        file_name2 = file_name
        while file_name2 == file_name:
            file_name2 = input(f'Enter the file name except for "{file_name}":\n')
        filename = os.path.join(project_dir, file_name2 + '.csv')

# Confirmation
if overwrite == 1:
    print(f'\033[33mThe file "{file_name2}.csv" will be saved after the following program\033[33m')
elif overwrite == 0:
    pass
else:
    print(f'\033[33mThe file "{file_name}.csv" will be saved after the following program\033[33m')

## 3rd: Run the Ketithley

In [ ]:
import pyvisa as visa
import time
import csv
import datetime
import sys
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from utilities import current_set




In [ ]:
## To check the Keithley address, use this code

from pymeasure.instruments import list_resources
list_resources()

## Make a combined figure

In [ ]:
import glob
import os
data_list = glob.glob(f'{project_dir}/*.csv')
csv_list = []
csv_list.sort()

for i, file in enumerate(data_list):
    csv_list.append(os.path.splitext(os.path.basename(file))[0])
    print(f'{i:02}: {os.path.splitext(os.path.basename(file))[0]}')

In [ ]:
plot_index = [0,1,2,3] # Chose the numbers
fig_name = input('filename')

plot_list = []
for i in range(len(plot_index)):
    plot_list.append(csv_list[plot_index[i]])

for i, file in enumerate(plot_list):
    print(f'{i}: {file}')
    
import pandas as pd

figure = plt.figure(figsize=(12,8))
plt.rcParams["font.size"] = 20
plt.xlabel('Time (s)')
plt.ylabel('Current (A)')

for file in plot_list:
    df = pd.read_csv(f'{project_dir}/{file}.csv')
    plt.plot(df['Time (s)'], df['Current (A)'], linestyle='-', marker='o', label = file)

# plt.legend()
plt.show()

# Create the project directory if it does not exist
fig_com_dir = f"{fig_dir}/combined"
make_folder(fig_com_dir)
fig_com_path = os.path.join(fig_com_dir, fig_name + '.png')
figure.savefig(fig_com_path, transparent = True)

In [ ]:
plot_index = [3,5,7,11,16,14] # Chose the numbers
fig_name = 'low-dose-pressureeffects-small-col-modified'

plot_list = []
for i in range(len(plot_index)):
    plot_list.append(csv_list[plot_index[i]])

for i, file in enumerate(plot_list):
    print(f'{i}: {file}')
    
import pandas as pd

figure = plt.figure(figsize=(12,8))
plt.rcParams["font.size"] = 20
plt.xlabel('Time (s)')
plt.ylabel('Current (A)')

for file in plot_list:
    df = pd.read_csv(f'{project_dir}/{file}.csv')
    x=df['Time (s)'].to_list()
    y=df['Current (A)'].to_list()
    x = [n-60 for n in x]
    y = [n/0.36*1e12 for n in y] #pA/cm2
    plt.plot(x, y, linestyle='-', marker='o', label = file)

plt.ylabel('J (pA/cm2)')
plt.xlim(0,60)
plt.ylim(-100,600)
plt.legend()
plt.show()

# Create the project directory if it does not exist
fig_com_dir = f"{fig_dir}/combined"
make_folder(fig_com_dir)
fig_com_path = os.path.join(fig_com_dir, fig_name + '.png')
figure.savefig(fig_com_path, transparent = True)